What we do here is load in the per gene file generated by the command **python compare_observed_and_expected_mut_burden.py casesFile mode** and reason on it

In [1]:
import pandas as pd
import os
import sys
from collections import Counter

In [2]:
sys.path.append('/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myUtils')
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util

In [4]:
pathPrefix = '/Users/friedman/Desktop/mnt'

In [4]:
reload(mutation_modeling_util)

<module 'mutation_modeling_util' from '/Users/friedman/Desktop/mnt/ifs/work/taylorlab/friedman/myUtils/mutation_modeling_util.pyc'>

In [18]:
df = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/simulatedMutationSummary.tsv')

In [19]:
oncogenicityDict = mutation_modeling_util.calculate_quadnuc_based_oncogenic_susceptibility_dict(df)


In [7]:
oncogenicityDictByGene = mutation_modeling_util.calculate_quadnuc_and_gene_based_oncogenic_susceptibility_dict(df)

considering the  25 th gene
considering the  50 th gene
considering the  75 th gene
considering the  100 th gene
considering the  125 th gene
considering the  150 th gene
considering the  175 th gene
considering the  200 th gene
considering the  225 th gene
considering the  250 th gene
considering the  275 th gene
considering the  300 th gene
considering the  325 th gene
considering the  350 th gene
considering the  375 th gene
considering the  400 th gene
considering the  425 th gene
considering the  450 th gene


In [27]:
allImpactMuts = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/all_impact_mutations_annotated_cohort.maf')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,36,39,85,136) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


**Now enumerate observed vs expected data for genes based on their type and cancer type**

we run the following on the cluster:
python compare_observed_and_expected_mut_burden.py /ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/idFiles/endometrialHypermutantIds.txt perGene

python compare_observed_and_expected_mut_burden.py /ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/idFiles/colorectalHypermutantIds.txt perGene

python compare_observed_and_expected_mut_burden.py /ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/idFiles/endometrialHypermutantIds.txt perGene outfile


In [71]:
endometrialDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/observedVsExpected/endometrialGenesObservedVsExpected.tsv')
colorectalDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/observedVsExpected/colorectalGenesObservedVsExpected.tsv')
gliomaDf = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/observedVsExpected/gliomaGenesObservedVsExpected.tsv')

In [72]:
cancerTypeImplicationDictBig = maf_analysis_utils.create_dictionary_mapping_genes_to_cancer_types_with_implication(allImpactMuts, pathPrefix, cancerTypes=set(['Glioma', 'Colorectal Cancer', 'Endometrial Cancer']), t=.1)
cancerTypeImplicationDictSmall = maf_analysis_utils.create_dictionary_mapping_genes_to_cancer_types_with_implication(allImpactMuts, pathPrefix, cancerTypes=set(['Glioma', 'Colorectal Cancer', 'Endometrial Cancer']), t=.025) 
for key, value in cancerTypeImplicationDictBig.items():
    curCTypeGenes = cancerTypeImplicationDictSmall[key]
    cancerTypeImplicationDictSmall[key] = curCTypeGenes - value


Endometrial Cancer
Glioma
Colorectal Cancer
Endometrial Cancer
Glioma
Colorectal Cancer


In [73]:
endometrialDf['cancer_type'] = 'Endometrial Cancer'
colorectalDf['cancer_type'] = 'Colorectal Cancer'
gliomaDf['cancer_type'] = 'Glioma'
combinedDf = pd.concat([endometrialDf, colorectalDf, gliomaDf])

In [74]:
combinedDf['geneMutType'] = combinedDf.apply(lambda row:
                                            'strongly_recurrent' if row['gene'] in cancerTypeImplicationDictBig[row['cancer_type']]
                                             else 'weakly_recurrent' if row['gene'] in cancerTypeImplicationDictSmall[row['cancer_type']]
                                            else 'not_recurrent', axis=1
                                            )

In [75]:
combinedDf['nCases'] = combinedDf['cancer_type'].apply(lambda x: 53 if x == 'Glioma'
                                                       else 261 if x == 'Colorectal Cancer' 
                                                       else 237)

In [ ]:
combinedDf['geneObsExpectedRatio'] = combinedDf.apply(lambda row: 1.0*(row['nObservedMuts']- row['nExpectedMuts'])/row['nCases'], axis=1)
combinedDf['geneDisplayName'] = combinedDf.apply(lambda row: 
                                                 row['gene'] if row['geneMutType'] == 'strongly_recurrent'
                                                 else row['gene'] if row['geneObsExpectedRatio'] > .25
                                                 else row['gene'] if row['geneObsExpectedRatio'] < -.1
                                                 else None,axis=1)


In [82]:
combinedDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/observedVsExpectedByGeneHypermutators.tsv', index=False, sep='\t')

**Add hotspot info into data**

In [ ]:
cntr = 0
d = pathPrefix + '/ifs/work/taylorlab/friedman/myAdjustedDataFiles/simulatedMafs/geneMutMafs/'
listOfDicts = []
for fPath in os.listdir(d):
    if 'v2' in fPath:
        print cntr
        cntr+=1
        
        gene = fPath.split('_')[0]
        df = pd.read_table(d + fPath)
        hotspots = df[df['is-a-hotspot'].notnull()]
        allPossibleQuadNucs = [firstNuc + change + lastNuc for firstNuc in ['A', 'T', 'C', 'G'] for change in ['CA', 'CG', 'CT', 'TA', 'TC', 'TG'] for lastNuc in ['A', 'T', 'C', 'G']]
        nHotspots = hotspots.shape[0]
        quadNucCounts = Counter(hotspots['quadNuc'])
        
        localD = {}
        for quadNuc in allPossibleQuadNucs:
            quadNucCount = 0
            if quadNuc in quadNucCounts:
                quadNucCount = quadNucCounts[quadNuc]
            localD[quadNuc + '_hotspot'] = quadNucCount
        localD['nHotspot'] = nHotspots
        localD['Hugo_Symbol'] = gene
        listOfDicts.append(localD)
        
hotspotDf = pd.DataFrame(listOfDicts)
            
        

In [41]:
df = pd.read_table(pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/simulatedMutationSummary.tsv')

In [45]:
hotspotDfCopy = hotspotDf.copy()

In [46]:
mergedDf = df.merge(hotspotDf, left_on='Hugo_Symbol', right_on='Hugo_Symbol')

In [48]:
mergedDf.to_csv('/Users/friedman/Desktop/workOffline/mutSimulationInfoIncludingHotspots.tsv', index=False, sep='\t')

**HOTSPOTS IN PANEL VS NOT ANALYSIS**
<br/><br/><br/><br/>

In [3]:
#offline instead of imports

import re

def enumerate_recurrently_mutated_tumor_supressors_and_oncogenes(cohortMaf, thresh=.1):
    tumorSupressors = set(['ERRFI1', 'ASXL2', 'PMAIP1', 'ACTG1', 'SUFU', 'FBXO11', 'MEN1', 'FAM58A', 'B2M', 'RB1', 'DUSP22', 'SESN1', 'GPS2', 'RAD51D', 'SMG1', 'CDC73', 'MAP3K1', 'SMARCB1', 'INPP4B', 'PARK2', 'SMAD4', 'CBFB', 'CDH1', 'PPP6C', 'SETDB1', 'SETDB2', 'NF2', 'CDKN2B', 'CDKN2C', 'CDKN2A', 'DDX3X', 'PIK3R1', 'BARD1', 'PDS5B', 'KLF4', 'SPRED1', 'VHL', 'SMAD2', 'PMS1', 'PMS2', 'SETD2', 'GATA3', 'TBL1XR1', 'MUTYH', 'SOCS1', 'FAM175A', 'ROBO1', 'ARID1B', 'ARID1A', 'TCF7L2', 'STK11', 'FOXA1', 'PTEN', 'FAT1', 'FAS', 'CYLD', 'MAX', 'SH2D1A', 'APC', 'NTHL1', 'CTCF', 'KDM5C', 'KMT2C', 'ZFHX3', 'FOXP1', 'PIGA', 'CDKN1B', 'CDKN1A', 'FUBP1', 'MSH2', 'ID3', 'TNFRSF14', 'TRAF3', 'EP400', 'BRIP1', 'ARID4A', 'ARID4B', 'XRCC2', 'DAXX', 'SDHAF2', 'ASXL1', 'AMER1', 'RASA1', 'EGR1', 'MST1', 'SOX17', 'RUNX1', 'PIK3R3', 'NCOR1', 'NF1', 'JAK1', 'PTPRD', 'CHEK2', 'CHEK1', 'SMC1A', 'TMEM127', 'STAG1', 'RAD51', 'TCF3', 'STAG2', 'ARID2', 'RAD50', 'RNF43', 'PARP1', 'BLM', 'CUX1', 'RECQL', 'RAD21', 'PTPN2', 'PTPN1', 'SLX4', 'INHA', 'PAX5', 'IRF1', 'TP53', 'HLA-A', 'IRF8', 'CBL', 'TOP1', 'SHQ1', 'PRDM1', 'NSD1', 'ATXN2', 'CREBBP', 'HDAC4', 'SESN2', 'PPP2R1A', 'EPHA7', 'ATM', 'EPHA3', 'POT1', 'SMAD3', 'MOB3B', 'TBX3', 'POLE', 'ATR', 'FANCD2', 'FH', 'BCORL1', 'SOX9', 'IKZF3', 'TSC1', 'TP63', 'MRE11A', 'SDHC', 'BTG1', 'POLD1', 'CIITA', 'SMC3', 'SAMHD1', 'RTEL1', 'ECT2L', 'PIK3R2', 'CRBN', 'FANCC', 'NBN', 'FANCA', 'HLA-B', 'RECQL4', 'DUSP4', 'ERCC2', 'FBXW7', 'TGFBR2', 'TGFBR1', 'MSH3', 'RBM15', 'TET1', 'TET3', 'SESN3', 'MGA', 'LTB', 'FOXL2', 'SH2B3', 'BCOR', 'HIST1H1D', 'ATRX', 'EP300', 'RAD51C', 'RAD51B', 'HIST1H1B', 'TNFAIP3', 'DICER1', 'ARID5B', 'LATS2', 'FOXO1', 'KEAP1', 'EZH2', 'SP140', 'NKX3-1', 'PBRM1', 'PALB2', 'CIC', 'BRCA1', 'DTX1', 'FLCN', 'SPEN', 'CD58', 'ERCC3', 'ERCC4', 'MSH6', 'BCL11B', 'BMPR1A', 'ERF', 'BRCA2', 'NOTCH2', 'EED', 'MITF', 'ELF3', 'SMARCA4', 'BBC3', 'ANKRD11', 'CEBPA', 'BCL2L11', 'AXIN2', 'AXIN1', 'CDK12', 'ESCO2', 'MLH1', 'SDHB', 'MED12', 'HNF1A', 'RYBP', 'ATP6V1B2', 'DNMT3B', 'KMT2B', 'KMT2A', 'DNMT3A', 'NFKBIA', 'TRAF5', 'KMT2D', 'SPOP', 'RBM10', 'P2RY8', 'TP53BP1', 'TSC2', 'KDM6A', 'EPCAM', 'PHOX2B', 'NPM1', 'BCL10', 'LATS1', 'HOXB13', 'ARID3A', 'PTPRT', 'PTPRS', 'INPPL1', 'NOTCH4', 'TET2', 'NOTCH1', 'CASP8', 'NOTCH3', 'GRIN2A', 'MAP2K4', 'WT1', 'BACH2', 'SDHA', 'BAP1', 'PTCH1', 'SDHD'])
    occurenceCounter, rankingDict, fractionalDict = enumerate_top_n_oncogenic_mutated_genes_across_cohort(cohortMaf, n=50)
    recurrentTumorSupressors = []
    recurrentOncogenes = []
    for key, value in fractionalDict.items():
        if value > thresh:
            if key in tumorSupressors:
                recurrentTumorSupressors.append(key)
            else:
                recurrentOncogenes.append(key) 
    return set(recurrentTumorSupressors), set(recurrentOncogenes)

def get_ids_by_hypermutant_status(hypermutantIdDir='/Users/friedman/Desktop/hypermutationStatusIdsOffline/', cancerType='', hypermutantStatus = 'Hypermutated'):
    cancerTypeAdj = re.sub(' ', '_', cancerType)
    path = os.path.join(hypermutantIdDir, cancerTypeAdj + '.tsv')
    df = pd.read_table(path)
    if hypermutantStatus == 'all':
        return set(df['Tumor_Sample_Barcode'])
    else:
        return set(df[df['hypermutantClassification'] == hypermutantStatus]['Tumor_Sample_Barcode'])

def create_reference_four_nuc(refTri, refAllele, altAllele, variantType):
    #properly invert when needed
    def invert_allele_and_ref_tri(altAllele):
        nucleotideDict = {'A': 'T', 'G': 'C', 'C':'G', 'T':'A'}
        return nucleotideDict[altAllele]

    if variantType != 'SNP': return None  #there is no reference trinuc for non snps
    if not isinstance(refTri, basestring): return None #if the ref tri is not a string we better return none
    if len(refTri) < 3: return None # if the ref tri is less than length 3 (at the end of an exon), we cant do anything
    refTri = str(refTri) #just make sure the ref tri is a string here to avoid funny business
    refAlleleFromReftri = refTri[1]
    alt = altAllele
    if refAlleleFromReftri != refAllele:
        alt = invert_allele_and_ref_tri(altAllele)
    quadNuc = refTri[:2] + alt + refTri[2]
    return quadNuc    
    
    

In [16]:
#FOR US TO DO OBSERVED VS EXPECTED
def quantify_quadnuc_hotspot_susceptibility_per_mutation(simMafData, relatedGenes=None):
    allBases = ['A', 'C', 'G', 'T']
    changes = ['CA', 'CG', 'CT', 'TA', 'TC', 'TG'] #format: 'CA' means a change from C>A
    allQuadNucs = [firstBase + change + lastBase for firstBase in allBases for change in changes for lastBase in allBases] #enumerate all 96 quadnucs for signatures
    
    #simulatedMutsRelated = simMafData[simMafData['Hugo_Symbol'].isin(relatedGenes)]
    #simulatedMutsUnrelated = simMafData[~simMafData['Hugo_Symbol'].isin(relatedGenes)]
    
    d = {}
    
    for quadNuc in allQuadNucs:
        nPossibleMuts = sum(simMafData[quadNuc]) - sum(simMafData[quadNuc + '_silent'])
        nPossibleHotspotMuts = sum(simMafData[quadNuc + '_hotspot'])
        #nPossibleMutsRelated = sum(simulatedMutsRelated[quadNuc]) - sum(simulatedMutsRelated[quadNuc + '_silent'])
        #nPossibleMutsUnrelated = sum(simulatedMutsUnrelated[quadNuc]) - sum(simulatedMutsUnrelated[quadNuc + '_silent'])
        #nPossibleHotspotMutRelated = sum(simulatedMutsRelated[quadNuc + '_hotspot'])
        #nPossibleHotspotMutUnrelated = sum(simulatedMutsUnrelated[quadNuc + '_hotspot'])
        #d[quadNuc + '_related_hotspot_rate'] = (1.0*nPossibleHotspotMutRelated)/(1.0*nPossibleMutsRelated)
        #d[quadNuc + '_unrelated_hotspot_rate'] = (1.0*nPossibleHotspotMutUnrelated)/(1.0*nPossibleMutsUnrelated)
        d[quadNuc + '_hotspot_rate'] = (1.0*nPossibleHotspotMuts)/(1.0*nPossibleMuts)
        
    return d

def calculate_quadnuc_based_oncogenic_susceptibility_dict(simMafData):
    d = {}
    allBases = ['A', 'C', 'G', 'T']
    changes = ['CA', 'CG', 'CT', 'TA', 'TC', 'TG'] #format: 'CA' means a change from C>A
    allQuadNucs = [firstBase + change + lastBase for firstBase in allBases for change in changes for lastBase in allBases] #enumerate all 96 quadnucs for signatures
    for quadNuc in allQuadNucs:
        nPossibleMuts = sum(simMafData[quadNuc]) - sum(simMafData[quadNuc + '_silent'])
        nPossibleOncogenicMuts = sum(simMafData[quadNuc + '_oncogenic'])
        d[quadNuc + '_oncogenic_rate'] = (1.0*nPossibleOncogenicMuts)/(1.0*nPossibleMuts)
    return d
    

In [4]:
def calculate_observed_and_expected_by_mut_type_hotspot(allMuts, ids, relatedGenes, hotspotSusceptibilityD):
    listOfDicts = []
    for case in set(ids):
        caseMuts = allMuts[allMuts['Tumor_Sample_Barcode'] == case]
        quadNucCounts = Counter(caseMuts['quadNuc'])
        nmut = caseMuts.shape[0]

        caseHotspotMuts = caseMuts[caseMuts['is-a-hotspot'] == 'Y']

        nHotspotMuts = caseHotspotMuts.shape[0]
        #nCaseRelatedHotspotMuts = caseHotspotMuts[caseHotspotMuts['Hugo_Symbol'].isin(relatedGenes)].shape[0]
        #nCaseUnrelatedHotspotMuts = caseHotspotMuts[~caseHotspotMuts['Hugo_Symbol'].isin(relatedGenes)].shape[0]

        #nExpectedHotspotsRelated = 0
        #nExpectedHotspotsUnrelated = 0
        nExpectedHotspots = 0
        for quadNuc, count in quadNucCounts.items():
            if quadNuc != None:
                nExpectedHotspots += count*hotspotSusceptibilityD[quadNuc + '_hotspot_rate']
                #nExpectedHotspotsRelated += count*hotspotSusceptibilityD[quadNuc + '_related_hotspot_rate']
                #nExpectedHotspotsUnrelated += count*hotspotSusceptibilityD[quadNuc + '_unrelated_hotspot_rate']

        listOfDicts.append({'Tumor_Sample_Barcode': case, 
                            #'nRelatedHotspotsObserved': nCaseRelatedHotspotMuts, 'nUnrelatedHotspotsObserved': nCaseUnrelatedHotspotMuts,
                            #'nHotspotsObserved': nCaseRelatedHotspotMuts + nCaseUnrelatedHotspotMuts,
                           #'nExpectedHotspotRelated': nExpectedHotspotsRelated, 'nExpectedHotspotsUnrelated':  nExpectedHotspotsUnrelated,
                            #'nExpectedHotspots': nExpectedHotspotsRelated + nExpectedHotspotsUnrelated,
                           'nHotspotObserved': nHotspotMuts, 'nHotspotExpected': nExpectedHotspots,
                            'Nmut': nmut})

    obsVsExpected = pd.DataFrame(listOfDicts)
    return obsVsExpected
    
    

def calculate_observed_and_expected_by_mut_type_oncogenic_version(allMuts, ids, relatedGenes, oncogenicSusceptibilityDict):
    listOfDicts = []
    cntr = 0
    for case in set(ids):
        
        if cntr %100 == 0: print cntr
        cntr += 1
        
        caseMuts = allMuts[(allMuts['Tumor_Sample_Barcode'] == case) & (allMuts['Variant_Type'] == 'SNP')]
        quadNucCounts = Counter(caseMuts['quadNuc'])
        nmut = caseMuts.shape[0]

        caseOncogenicMuts = caseMuts[caseMuts['oncogenic'].notnull()]
        nCaseOncogenicMuts = caseOncogenicMuts.shape[0]
        
        nExpectedOncogenic = 0
        for quadNuc, count in quadNucCounts.items():
            if quadNuc != None:
                nExpectedOncogenic += count*oncogenicSusceptibilityDict[quadNuc + '_oncogenic_rate']
        listOfDicts.append({'Tumor_Sample_Barcode': case, 
                            'nOncogenicObserved': nCaseOncogenicMuts,
                            'nExpectedOncogenic': nExpectedOncogenic,
                           'Nmut': nmut})

    obsVsExpected = pd.DataFrame(listOfDicts)
    return obsVsExpected
    

In [8]:
allImpactMuts = pd.read_table('/Users/friedman/Desktop/workOffline/all_impact_mutations_annotated_cohort.maf')

/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (13,36,39,85,136) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
#do it with this later
#enumerate_recurrently_mutated_tumor_supressors_and_oncogenes(cohortMaf, thresh=.1)
endometrialCancerGenes = set(['POLE', 'MLH1', 'MSH2', 'MSH6', 'PMS2','PTEN', 'BCOR', 'PIK3CA', 'ARID1A', 'TP53', 'PIK3R1', 'FBXW7', 'KRAS', 'CTNNB1', 'PPP2R1A', 'FGFR'])
colorectalCancerGenes = set(['APC', 'PIK3CA', 'TP53', 'KRAS', 'SMAD4', 'ARID1A', 'PTEN', 'FBXW7', 'TCF7L2', 'SOX9', 'BRAF', 'POLE', 'MLH1', 'MSH2', 'MSH6', 'PMS2'])
gliomaGenes = set(['TP53', 'TERT', 'PIK3CA', 'NF1', 'IDH1', 'PTEN', 'ATRX', 'EGFR', 'CIC', 'BRAF', 'KRAS'])


In [ ]:
endometrialIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Endometrial Cancer', hypermutantStatus = 'Hypermutated')
colorectalIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Colorectal Cancer', hypermutantStatus = 'Hypermutated')
gliomaIds = analysis_utils.get_ids_by_hypermutant_status(hypermutantIdDir=pathPrefix +'/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds', cancerType='Glioma', hypermutantStatus = 'Hypermutated')


In [98]:
endometrialCancerHyperIds = get_ids_by_hypermutant_status(cancerType='Endometrial Cancer', hypermutantStatus = 'Hypermutated')
colorectalCancerHyperIds = get_ids_by_hypermutant_status(cancerType='Colorectal Cancer', hypermutantStatus = 'Hypermutated')
gliomaCancerHyperIds = get_ids_by_hypermutant_status(cancerType='Glioma', hypermutantStatus = 'Hypermutated')

endometrialCancerIds = get_ids_by_hypermutant_status(cancerType='Endometrial Cancer', hypermutantStatus = 'all')
colorectalCancerIds = get_ids_by_hypermutant_status(cancerType='Colorectal Cancer', hypermutantStatus = 'all')
gliomaCancerIds = get_ids_by_hypermutant_status(cancerType='Glioma', hypermutantStatus = 'all')


In [9]:
allImpactMuts['quadNuc'] = allImpactMuts.apply(lambda row: mutationSigUtils.create_reference_four_nuc(row['Ref_Tri'], row['Reference_Allele'], row['Tumor_Seq_Allele2'], row['Variant_Type']), axis=1)
#endometrialCancerHyperMuts = allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(endometrialCancerHyperIds)]
#endometrialCancerHotspots = endometrialCancerHyperMuts[endometrialCancerHyperMuts['is-a-hotspot'] == 'Y']

In [18]:
simulatedDataSummary = pd.read_table('/Users/friedman/Desktop/workOffline/mutSimulationInfoIncludingHotspots.tsv')

In [19]:
hotspotSusceptibilityD = quantify_quadnuc_hotspot_susceptibility_per_mutation(simulatedDataSummary)
oncogenicityDict = calculate_quadnuc_based_oncogenic_susceptibility_dict(simulatedDataSummary)


In [178]:
colorectalDf = calculate_observed_and_expected_by_mut_type_hotspot(allImpactMuts, colorectalCancerIds, colorectalCancerGenes, hotspotSusceptibilityD)
endometrialDf = calculate_observed_and_expected_by_mut_type_hotspot(allImpactMuts, endometrialCancerIds, endometrialCancerGenes, hotspotSusceptibilityD)
gliomaDf = calculate_observed_and_expected_by_mut_type_hotspot(allImpactMuts, gliomaCancerIds, gliomaGenes, hotspotSusceptibilityD)

#colorectalDf = calculate_observed_and_expected_by_mut_type_oncogenic_version(allImpactMuts, colorectalCancerIds, colorectalCancerGenes, oncogenicityDict)
#endometrialDf = calculate_observed_and_expected_by_mut_type_oncogenic_version(allImpactMuts, endometrialCancerIds, endometrialCancerGenes, oncogenicityDict)
#gliomaDf = calculate_observed_and_expected_by_mut_type_oncogenic_version(allImpactMuts, gliomaCancerIds, gliomaGenes, oncogenicityDict)



In [179]:
colorectalDf['cancer_type'] = 'Colorectal Cancer'
endometrialDf['cancer_type'] = 'Endometrial Cancer'
gliomaDf['cancer_type'] = 'Glioma'

combinedDf = pd.concat([colorectalDf, endometrialDf, gliomaDf])



In [180]:
combinedDf.to_csv('/Users/friedman/Desktop/WORK/dataForLocalPlotting/obsVsExpectedHotspots.tsv', index=False, sep='\t')




**THIS SECTION IS CURRENTLY FOR US JUST TO SHOW THE DIFFERENT HOTSPOT SUSCEPTIBILITIES AND OBSERVEDS**

In [5]:
def count_n_hotspots_or_oncogenic_per_case(cohortMaf, label, mode='hotspot'):
    
    mafOfInterest = None
    
    if mode == 'hotspot':
        mafOfInterest = cohortMaf[cohortMaf['is-a-hotspot'] == 'Y']
    elif mode == 'oncogenic':
        mafOfInterest = cohortMaf[(cohortMaf['oncogenic'].notnull()) & (cohortMaf['Variant_Type'] == 'SNP')]

        
    countsD = dict(mafOfInterest['Tumor_Sample_Barcode'].value_counts())
    for case in set(cohortMaf['Tumor_Sample_Barcode']):
        if case not in countsD:
            countsD[case] = 0
    
    nmutD = dict(cohortMaf['Tumor_Sample_Barcode'].value_counts())
    for case in set(cohortMaf['Tumor_Sample_Barcode']):
        if case not in nmutD:
            nmutD[case] = 0
            
    listOfDicts = []
    for key, values in nmutD.items():
        nmut = nmutD[key]
        nHotspotOrOnc = countsD[key]
        hotspotOrOncFrac = 0
        if nmut != 0:
            hotspotOrOncFrac = (1.0*nHotspotOrOnc)/nmut
        
        localD = {'id': key, 'class': label}
        if mode == 'hotspot':
            localD['hotspotFrac'] = hotspotOrOncFrac
        elif mode == 'oncogenic':
            localD['oncogenicFrac'] = hotspotOrOncFrac
        
        listOfDicts.append(localD)
        
    return pd.DataFrame(listOfDicts)


#converts the dictionary of hypermutation
def convert_hotspot_or_oncogenic_dict_to_df_format(d, mode='hotspot'):
    listOfDicts = []
    for key, value in d.items():
        localD = {
            'class': 'theoreticalSusceptibility',
            'id': key
        }
        if mode == 'hotspot':
            localD['hotspotFrac'] = value
        elif mode == 'oncogenic':
            localD['oncogenicFrac'] = value
        listOfDicts.append(localD)
    return pd.DataFrame(listOfDicts)
        

In [12]:
endometrialHyperIds = analysis_utils.get_ids_by_hypermutant_status(cancerType='Endometrial Cancer', hypermutantStatus = 'Hypermutated', hypermutantIdDir= pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds')
colorectalHyperIds = analysis_utils.get_ids_by_hypermutant_status(cancerType='Colorectal Cancer', hypermutantStatus = 'Hypermutated', hypermutantIdDir= pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds')
gliomaHyperIds = analysis_utils.get_ids_by_hypermutant_status(cancerType='Glioma', hypermutantStatus = 'Hypermutated', hypermutantIdDir= pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds')

endometrialNormalIds = analysis_utils.get_ids_by_hypermutant_status(cancerType='Endometrial Cancer', hypermutantStatus = 'Normal', hypermutantIdDir= pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds')
colorectalNormalIds = analysis_utils.get_ids_by_hypermutant_status(cancerType='Colorectal Cancer', hypermutantStatus = 'Normal', hypermutantIdDir= pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds')
gliomaNormalIds = analysis_utils.get_ids_by_hypermutant_status(cancerType='Glioma', hypermutantStatus = 'Normal', hypermutantIdDir= pathPrefix + '/ifs/work/taylorlab/friedman/hypermutationAnalysisProj/projectDataAndConfigFiles/hypermutationStatusIds')


In [50]:
m = 'oncogenic'
#m = 'hotspot'
endoNormalDf = count_n_hotspots_or_oncogenic_per_case(allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(endometrialNormalIds)], 'endometrialNormal', mode=m)
colorectalNormalDf = count_n_hotspots_or_oncogenic_per_case(allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(colorectalNormalIds)], 'colorectalNormal', mode=m)
gliomaNormalDf = count_n_hotspots_or_oncogenic_per_case(allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(gliomaNormalIds)], 'gliomaNormal', mode=m)

endoHyperDf = count_n_hotspots_or_oncogenic_per_case(allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(endometrialHyperIds)], 'endometrialHyper', mode=m)
colorectalHyperDf = count_n_hotspots_or_oncogenic_per_case(allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(colorectalHyperIds)], 'colorectalHyper', mode=m)
gliomaHyperDf = count_n_hotspots_or_oncogenic_per_case(allImpactMuts[allImpactMuts['Tumor_Sample_Barcode'].isin(gliomaHyperIds)], 'gliomaHyper', mode=m)


In [21]:
hypotheticalDfHotspots = convert_hotspot_or_oncogenic_dict_to_df_format(hotspotSusceptibilityD, mode = 'hotspot')
hypotheticalDfOncogenic = convert_hotspot_or_oncogenic_dict_to_df_format(oncogenicityDict, mode = 'oncogenic')

In [51]:
combinedDf = pd.concat([endoNormalDf, colorectalNormalDf, gliomaNormalDf,
                       endoHyperDf, colorectalHyperDf, gliomaHyperDf, hypotheticalDfHotspots, hypotheticalDfOncogenic])
combinedDf['simplifiedClass'] = combinedDf['class'].apply(lambda x: 'Hyper' if 'Hyper' in x 
                                                         else 'Normal' if 'Normal' in x else x)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [52]:
#combinedDf.to_csv('/Users/friedman/Desktop/WORK/hotspotFractionOfMutationBurden.tsv', index=False, sep='\t')
combinedDf.to_csv('/Users/friedman/Desktop/WORK/oncogenicFractionOfMutationBurden.tsv', index=False, sep='\t')

In [38]:
#max(combinedDf[combinedDf['class'] == 'theoreticalSusceptibility']['hotspotFrac'])
#combinedDf[combinedDf['class'] == 'theoreticalSusceptibility']


print max(hypotheticalDfHotspotsSigs['hotspotFrac'])


0.00417333686624


**Give information about mutation signatures and susceptibility**

In [31]:
def convert_quadnuc_df_to_mut_sigs_df(quadNucDf, colName):
    quadNucDict = dict(zip(quadNucDf['id'].apply(lambda x: x[:4]), quadNucDf[colName]))
    spectraD = mutationSigUtils.convert_spectrum_file_to_dict_of_dicts(spectrumFile=pathPrefix + '/ifs/work/taylorlab/friedman/noahFirstProject/signature_sig_copy/mutation-signatures/Stratton_signatures30.txt')
    d = {}
    for signature in spectraD.keys():
        s = 0
        for quadNuc, frac in spectraD[signature].items():
            s += frac*quadNucDict[quadNuc]
        d[signature] = s
    return d
    
    

In [32]:
dHotspot = convert_quadnuc_df_to_mut_sigs_df(hypotheticalDfHotspots, 'hotspotFrac')
dOncogenic = convert_quadnuc_df_to_mut_sigs_df(hypotheticalDfOncogenic, 'oncogenicFrac')

In [33]:
hypotheticalDfHotspotsSigs = convert_hotspot_or_oncogenic_dict_to_df_format(dHotspot, mode = 'hotspot')
hypotheticalDfOncogenicSigs = convert_hotspot_or_oncogenic_dict_to_df_format(dOncogenic, mode = 'oncogenic')

In [93]:
#combinedDf = pd.concat([endoNormalDf, colorectalNormalDf, gliomaNormalDf,
#                       endoHyperDf, colorectalHyperDf, gliomaHyperDf, hypotheticalDfHotspotsSigs])
combinedDf = pd.concat([endoNormalDf, colorectalNormalDf, gliomaNormalDf,
                       endoHyperDf, colorectalHyperDf, gliomaHyperDf, hypotheticalDfOncogenicSigs])


combinedDf['simplifiedClass'] = combinedDf['class'].apply(lambda x: 'Hyper' if 'Hyper' in x 
                                                         else 'Normal' if 'Normal' in x else x)
combinedDf.to_csv('/Users/friedman/Desktop/WORK/hotspotFractionOfMutationBurden.tsv', index=False, sep='\t')

**Info for hotspot frac amongst all observed mutations only** <br/> <br/>

In [95]:
allImpactMuts['variantUUID'] = allImpactMuts.apply(lambda row: str(row['Start_Position']) + '_' + str(row['Tumor_Seq_Allele2']), axis=1)


In [104]:
allImpactMutsUnique = allImpactMuts.drop_duplicates(subset=['variantUUID'])

In [49]:

max(hypotheticalDfOncogenic[hypotheticalDfOncogenic['class'] == 'theoreticalSusceptibility']['oncogenicFrac'])
#hypotheticalDfOncogenic


#print hypotheticalDfOncogenic[(hypotheticalDfOncogenic['class'] == 'theoreticalSusceptibility') & (hypotheticalDfOncogenic['oncogenicFrac'] > .2)]

#hypotheticalDfOncogenicSigs

0.2030494302811706

In [108]:
for quadNuc in set(allImpactMutsUnique['quadNuc']):
    quadNucMuts = allImpactMutsUnique[allImpactMutsUnique['quadNuc'] == quadNuc]
    print quadNuc, (1.0*quadNucMuts[quadNucMuts['oncogenic'].notnull()].shape[0])/quadNucMuts.shape[0]

GTAC 0.135746606335
GTAA 0.258741258741
GTAG 0.145488029466
GTAT 0.0812641083521
TTAG 0.518438177874
TTAC 0.309771309771
TTAA 0.57650273224
TTAT 0.176356589147
GCTG 0.0684299554052
GCTC 0.0440528634361
GCTA 0.27494199536
ATAA 0.145679012346
ATAC 0.119225037258
CCAT 0.137020566982
ATAG 0.136212624585
GCTT 0.0504396112911
CCAA 0.121383647799
CCAC 0.0744732974032
ATAT 0.112012987013
CCAG 0.0955357142857
TCTT 0.0682471264368
ACTT 0.0955823293173
TCTG 0.0890282131661
CTCA 0.251644736842
CTCC 0.0518518518519
TCTC 0.0432801822323
TCTA 0.190149103702
ACTG 0.0807099091485
CTAG 0.208181818182
CTAC 0.092084006462
CTAA 0.392971246006
GTGA 0.404958677686
GTGC 0.0909090909091
GTGG 0.149877149877
ACGA 0.158974358974
ACGC 0.256916996047
ACGG 0.177130044843
GTGT 0.0593220338983
TTCT 0.0435540069686
TTCG 0.0586552217454
TCAC 0.29542042042
TTCA 0.0742268041237
TTCC 0.0524861878453
ATCT 0.0459126539754
CCGG 0.111545988258
CTGC 0.0598086124402
CCGC 0.0675944333996
CCGA 0.0914634146341
ATCG 0.0765550239234


ZeroDivisionError: float division by zero